### Step 3.a - Walking Model

We have built a fairly sophisticated walking model for the Yemen analysis, where walk times are adjusted based on the relative elevation and slope of direction of travel. This scripts sets up such a graph. 

It requires the user to have all of the SRTM tiles that relate to the country saved locally on their PC / workspace so that the various functions can call them when necessary.

The tiles can be downloaded here: http://dwtkns.com/srtm30m/ 

Import the usual suspects

In [ ]:
import sys, os, time
import geopandas as gpd
import pandas as pd
import rasterio as rt
import numpy as np
from affine import Affine
from shapely.geometry import Point, LineString
from functools import partial
import pyproj
from shapely.ops import transform
from shapely.geometry import box
from shapely.wkt import loads
from rasterio import features
import networkx as nx
sys.path.append(r'C:\Users\charl\Documents\GitHub\GOST_PublicGoods\GOSTNets\GOSTNets')
sys.path.append(r'C:\Users\charl\Documents\GitHub\GOST')
import GOSTnet as gn
import importlib

### Initial Graph Read In

This graph will usually be the 'final product' graph - after any adjustments for conflict, for example. 

In [2]:
bpth = r'C:\Users\charl\Documents\GOST\Yemen\graphtool'
G = nx.read_gpickle(os.path.join(bpth, 'G_salty_time_conflict_adj.pickle'))

### Walk all tiles, find path

pth is the folder containing all of the SRTM tiles, downloaded in .hgt format, which rasterio can natively read.

In [3]:
pth = r'C:\Users\charl\Documents\GOST\Yemen\SRTM\high_res'
tiles = []

# pick up any files that end in extension 'hgt', add to tiles list.
for root, folder, files in os.walk(pth):
    for f in files:
        if f[-3:] == 'hgt':
            tiles.append(f[:-4])

### Load dictionary of tiles 

Here, we generate a dictionary, arrs, which is the loaded versions of each tile, where t is the filename.

In [4]:
arrs = {}
for t in tiles:
    fpath = r'C:\Users\charl\Documents\GOST\Yemen\SRTM\high_res\{}.hgt\{}.hgt'.format(t, t)
    arrs[t] = rt.open(fpath, 'r')

### Add correct tile for each node into Graph Data Dictionary

In order to keep the matching on of elevation values to nodes nice and efficient, we devise a strategy whereby we assign the tile code to each node. The tiles are divided up by latitude and longitude coordinate; therefore, we can take the first two digits of the 'x' and 'y' attributes of each node to generate their tile 'code', attached as a new node attribute, 'code'. 

We also create a list of the unique tile codes, so we know if we have missed any when we downloaded the tiles.

In [5]:
uniques = []
for u, data in G.nodes(data = True):
    E = str(data['x'])[:2]
    N = str(data['y'])[:2]
    data['code'] = 'N{}E0{}'.format(N, E)
    uniques.append('N{}E0{}'.format(N, E))
    
# create unique list of tilecodes
unique_codes = list(set(uniques))

### Match on High Precision Elevation

Now, for each tile, we go through and read the relevant values on to the nodes where their tilecode matches the existing tile's code. 

In [6]:
property_name = 'elevation'

# for each unique tile code...
for code in unique_codes:
    
    # make a blank dictionary of nodes
    list_of_nodes = {}
    
    # cycle through all nodes, and if their code is the same as current code, add their x and y to the dictionary of nodes
    for u, data in G.nodes(data=True):
        if data['code'] == code:
            list_of_nodes.update({u:(data['x'], data['y'])})
            
    # now, we intersect these nodes with the raster, after doing an intersection
    dataset = arrs[code]
    b = dataset.bounds
    
    # make a bounding box for that tile
    datasetBoundary = box(b[0], b[1], b[2], b[3])
    
    # iterate through to make sure of intersection...
    selKeys = []
    selPts = []
    for key, pt in list_of_nodes.items():
        if Point(pt[0], pt[1]).intersects(datasetBoundary):
            selPts.append(pt)
            selKeys.append(key)
    
    # then use the .sample method to get the elevation of the intersecting points
    raster_values = list(dataset.sample(selPts))
    raster_values = [x[0] for x in raster_values]

    # generate new dictionary of {node ID: raster values}
    ref = dict(zip(selKeys, raster_values))
    
    # attach this data to the nodes with that u ID. 
    for u, data in G.nodes(data=True):
        if u in ref.keys():
            data[property_name] = ref[u]

### Add on low precision elevation for missed nodes

We repeat the above process, but this time we intersect with the SRTM 30 arc second product with filled voids. We only take the value from this raster if we have a no data value from the last step - i.e. the elevation is below -50m. 

This ensures we do not have any void values for the 'elevation' field for any nodes

In [7]:
for u, data in G.nodes(data=True):
    if data['elevation'] < -50:
        list_of_nodes.update({u:(data['x'], data['y'])})
        
tifpath = r'C:\Users\charl\Documents\GOST\Yemen\SRTM\clipped'
t = r'clipped_e20N40.tif'
tt = os.path.join(tifpath, t)
dataset = rt.open(tt, 'r')
b = dataset.bounds
datasetBoundary = box(b[0], b[1], b[2], b[3])
selKeys = []
selPts = []
for key, pt in list_of_nodes.items():
    if Point(pt[0], pt[1]).intersects(datasetBoundary):
        selPts.append(pt)
        selKeys.append(key)
raster_values = list(dataset.sample(selPts))
raster_values = [x[0] for x in raster_values]

ref = dict(zip(selKeys, raster_values))
for u, data in G.nodes(data=True):
    if u in ref.keys():
        data['elevation'] = ref[u]

### For the remaining mistakes, set elevation to 0

If the above two processes STILL fail to find an elevation for every node, set it to 0 because God help that node. 

In [8]:
for u, data in G.nodes(data=True):
    if data['elevation'] < 0:
        data['elevation'] = 0

### Generate dictionary of elevations for each node

Having found an appropriate elevation value, we load this into a dictionary for referencing later called elev_dict. 

In [9]:
elev_dict = {}
for u, data in G.nodes(data = True):
    if 'elevation' in data.keys(): 
        if data['elevation'] < 0:
            elev_dict[u] = 0
        else:
            elev_dict[u] = data['elevation']
    else:
        elev_dict[u] = 0

We run a quick Sense check - by printing out the max and minimum elevation recorded

In [10]:
max(list(elev_dict.values())), min(list(elev_dict.values()))

(3652, 0)

### Generate New Walking Data in existing edges

Now, we can do the relatively easy bit of adding in walking travel times. We use Tobler's Hiking function to define the walking speed with respect to the incline ratio, which is itself easy to calculate now that we know the elevation of the start node and end node of every node in our graph. 

Tobler's hiking function: https://en.wikipedia.org/wiki/Tobler%27s_hiking_function

In [11]:
def speed(incline_ratio, max_speed):
    walkspeed = max_speed * np.exp(-3.5 * abs(incline_ratio + 0.05)) 
    return walkspeed

# absolute max walkspeed, if heading downhill per Tobler's function
max_walkspeed = 6

# we set a floor speed to prevent ridiculous values
min_speed = 0.1

for u, v, data in G.edges(data = True):
    
    # pick up the start and end elevations
    data['elev_start'] = elev_dict[u]
    data['elev_end'] = elev_dict[v]
    
    # if the edge length is 0, the others will also be 0. Special case. 
    if data['length'] == 0:
        data['walkspeed'] = 0
        data['walk_time'] = 0
    else:
        # identify change in elevation 
        delta_elevation = data['elev_end'] - data['elev_start']
        
        # determine the incline ratio
        incline_ratio = delta_elevation / data['length']
        
        # apply the speed function to get walkspeed
        speed_kmph = speed(incline_ratio = incline_ratio, max_speed = max_walkspeed)
        
        # apply lower bound to values
        speed_kmph = max(speed_kmph, min_speed)
        
        # set as walkspeed attirbute for that edge
        data['walkspeed'] = speed_kmph
        
        # work out travel time
        data['walk_time'] = data['length'] / 1000 * 3600 / speed_kmph

### Save Down

We save down the completed graph. 

In [12]:
gn.save(G, 'walk_graph', bpth, nodes = False, edges = False)